In [2]:
import gensim
import os
import collections
import smart_open
import random

/home/jw/anaconda3/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [3]:
train_file = '/home/jw/project/spark/doc2vec/test.txt'

In [4]:

def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument([gensim.utils.any2unicode(i) for i in line.split()], [i])
                

In [5]:
gensim.utils.any2utf8?

In [6]:
train_corpus = list(read_corpus(train_file))

In [7]:
gensim.models.doc2vec.TaggedDocument??

In [8]:
train_corpus[0]

TaggedDocument(words=['mmsc.monternet.com', 'conf.3g.qq.com', 'qlogo1.store.qq.com', 'm.qpic.cn', 'wspeed.qq.com', 'activeqq.3g.qq.com', 'pub.idqqimg.com', '3gimg.qq.com', 'activeqq.3g.qq.com', '3gimg.qq.com', 'm.qpic.cn', 'plus.xiangji.qq.com', 'm.qpic.cn', 'plus.xiangji.qq.com', 'm.qpic.cn', 'plus.xiangji.qq.com', 'm.qpic.cn', 'conf.3g.qq.com', 'm.qpic.cn', 'p.store.qq.com', 'm.qpic.cn', 'plus.xiangji.qq.com', 'm.qpic.cn', 'conf.3g.qq.com', 'm.qpic.cn', 'p.store.qq.com', 'm.qpic.cn', 'plus.xiangji.qq.com', 'm.qpic.cn', 'monitor.uu.qq.com', 'm.qpic.cn', 'plus.xiangji.qq.com', 'm.qpic.cn', 'plus.xiangji.qq.com', '10.0.0.172', 'wspeed.qq.com', 'monitor.uu.qq.com', 'wspeed.qq.com', 'p.store.qq.com', '117.135.171.215', 'wspeed.qq.com', 'info60.z.qq.com', '120.203.230.182', 'v.gdt.qq.com', 'a258.photo.store.qq.com', 'a127.photo.store.qq.com', 'a258.photo.store.qq.com', '120.203.230.182', 'blog60.z.qq.com', 'v.gdt.qq.com', 'blog60.z.qq.com', '120.203.230.182', 'qzonestyle.gtimg.cn', '120.20

In [65]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55, workers=100)


In [66]:
model.build_vocab(train_corpus)

In [67]:
%time model.train(train_corpus)

CPU times: user 29min 10s, sys: 21.8 s, total: 29min 32s
Wall time: 3min 59s


255132460

In [70]:
model.infer_vector(['www.baidu.com'])

array([-0.0051687 ,  0.11520504, -0.03220525,  0.07329584, -0.02922823,
       -0.05916709, -0.08949592,  0.16703837,  0.17712016, -0.19990048,
        0.03526289, -0.07745354, -0.20547038, -0.09928478, -0.02248691,
        0.06048445, -0.05257897, -0.06377915, -0.04518294,  0.08737437,
       -0.01750471, -0.00060964, -0.06784464,  0.09676439,  0.16293456,
        0.20332322, -0.12239085,  0.08489628, -0.23380667,  0.00552376,
        0.08219492, -0.16552986, -0.05104955, -0.10484041,  0.08562284,
       -0.05232669,  0.08403423, -0.05995628,  0.08196908, -0.08162282,
        0.0599817 ,  0.12893651, -0.04566042,  0.06061307,  0.10993739,
        0.05293578, -0.00183644,  0.13404529,  0.02351481, -0.06250689], dtype=float32)

In [69]:
model.infer_vector(['only'])

array([ -9.71526955e-04,  -6.25042652e-04,   6.48722798e-03,
        -5.97501220e-03,   6.19377941e-04,   1.16862159e-03,
         7.70496484e-03,   6.79999869e-03,  -1.19464588e-03,
        -8.53012409e-03,   3.67454626e-03,   9.42977238e-03,
         6.54420815e-03,  -8.19337438e-04,  -9.62834060e-03,
         7.76772154e-03,   2.18170951e-03,   9.24124010e-03,
         9.84979142e-03,   9.02022410e-04,  -2.31309430e-04,
        -9.09298033e-05,   5.03535150e-03,  -3.02261906e-03,
        -1.98686984e-03,   4.75452421e-03,   8.27871356e-03,
         7.64060114e-03,   9.09098331e-03,   8.56771693e-03,
         3.24668270e-03,  -6.17854716e-03,   4.84256493e-03,
         7.34344078e-03,  -4.45673149e-03,  -6.52507879e-03,
         6.55823248e-03,   1.68913894e-03,   9.89754219e-03,
         6.44636247e-03,   2.80873314e-03,  -9.70168319e-03,
        -8.25118646e-03,  -7.15151802e-03,  -9.12653282e-03,
         3.61963036e-03,   1.37391000e-03,   2.30686064e-03,
         2.46210839e-03,

In [78]:
ranks = []
second_ranks = []
for doc_id in range(100):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=2)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    second_ranks.append(sims[1])

In [79]:
collections.Counter(ranks)

Counter({0: 99, 1: 1})

In [80]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (99): «init-p01st.push.apple.com monitor.uu.qq.com conf.3g.qq.com monitor.uu.qq.com configsvr.msf.3g.qq.com jsync.3g.qq.com pub.idqqimg.com pnewsapp.tc.qq.com monitor.uu.qq.com m.qpic.cn ugc.qpic.cn m.qpic.cn ugc.qpic.cn m.qpic.cn short.weixin.qq.com mmsns.qpic.cn init-p01st.push.apple.com fishingjoy.punchbox.org uapi.punchbox.org fishingjoy.punchbox.org fishingjoy2.appget.cn data.flurry.com fishingjoy.punchbox.org fishingjoy2.appget.cn sandbox.gc.apple.com fishingjoy2.appget.cn data.flurry.com down.qq.com 120.209.135.192 cgi.connect.qq.com cgi.qplus.com monitor.uu.qq.com configsvr.msf.3g.qq.com conf.3g.qq.com monitor.uu.qq.com pub.idqqimg.com kiss.3g.qq.com pub.idqqimg.com kiss.3g.qq.com qzs.qq.com rd.go.10086.cn ax.init.itunes.apple.com 120.209.135.192 monitor.uu.qq.com opensdk.uu.qq.com q1.qlogo.cn appimg1.3g.qq.com monitor.uu.qq.com web.g.qq.com iyouxi.vip.qq.com gamecenter.qq.com web.g.qq.com imgcache.qq.com isdspeed.qq.com web.g.qq.com q4.qlogo.cn m.qpic.cn init-p01st.pu

In [96]:
gensim.models.ldamulticore.LdaMulticore(corpus=[i[0] for i in train_corpus[:100]], alpha= 0.001, eta = 0.001,id2word=None, num_topics=10, offset = 50.0, chunksize=10000)

ValueError: too many values to unpack (expected 2)

In [9]:
gensim.models.ldamulticore.LdaMulticore?

In [10]:
[i[0] for i in train_corpus[:2]]

[['mmsc.monternet.com',
  'conf.3g.qq.com',
  'qlogo1.store.qq.com',
  'm.qpic.cn',
  'wspeed.qq.com',
  'activeqq.3g.qq.com',
  'pub.idqqimg.com',
  '3gimg.qq.com',
  'activeqq.3g.qq.com',
  '3gimg.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'conf.3g.qq.com',
  'm.qpic.cn',
  'p.store.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'conf.3g.qq.com',
  'm.qpic.cn',
  'p.store.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'monitor.uu.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  '10.0.0.172',
  'wspeed.qq.com',
  'monitor.uu.qq.com',
  'wspeed.qq.com',
  'p.store.qq.com',
  '117.135.171.215',
  'wspeed.qq.com',
  'info60.z.qq.com',
  '120.203.230.182',
  'v.gdt.qq.com',
  'a258.photo.store.qq.com',
  'a127.photo.store.qq.com',
  'a258.photo.store.qq.com',
  '120.203.230.182',
  'blog60.z.qq.com',
  'v.gdt